In [57]:
import requests as r
import pymysql as mysql
import datetime
import time

cnx= {'host':'chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com',
  'username': 'chimpdata',
  'password': 'godofdata01',
  'db': 'chimpdata'}



In [58]:
key="9a0d6c52a1fc405f8d02e5c08d09a2df"
url="http://api.football-data.org/v1/soccerseasons/396/teams"
headers = {"X-Auth-Token": key}
d1_id=396

In [68]:
import pymysql as mysql

class league_match(object):
    
    def __init__(self,journey,homeTeam,homeTeam_id,awayTeam,awayTeam_id,date,goalsAwayTeam,goalsHomeTeam,league_id):
        self.journey=journey
        self.homeTeam=homeTeam
        self.awayTeam=awayTeam
        self.homeTeam_id=homeTeam_id
        self.awayTeam_id=awayTeam_id
        self.date=date
        self.goalsAwayTeam=goalsAwayTeam
        self.goalsHomeTeam=goalsHomeTeam
        self.league_id=league_id
        self.index=str(journey)+str(homeTeam_id)+str(awayTeam_id)+str(league_id)

class league(object):
    
    def __init__(self,league_id):
        
        self.clubs={}
        self.matchs=[]
        self.league_id=league_id
        self.get_list_team()
        self.get_matchs()
        
    def add_club_name(self,club_name,club_id):
        
        self.clubs[club_name]=club_id
        
        
    def add_match(self,match):
        
        self.matchs.append(match)
        
        
    def get_list_team(self):
        
        key="9a0d6c52a1fc405f8d02e5c08d09a2df"
        url="http://api.football-data.org/v1/soccerseasons/"+str(self.league_id)+"/teams"
        headers = {"X-Auth-Token": key}
        data=r.get(url,headers=headers).json()
        
        for d in data["teams"]:
            id_=d["_links"]["self"]["href"][-3:]
            league.add_club_name(self,club_name=d["name"],club_id=id_)
            
       
    def get_matchs(self):
        
        key="9a0d6c52a1fc405f8d02e5c08d09a2df"
        url='http://api.football-data.org/v1/soccerseasons/'+str(self.league_id)+'/fixtures'
        headers = {"X-Auth-Token": key}
        data=r.get(url,headers=headers)
        data=data.json()
        for d in data['fixtures']:
            
            if d["status"]=="FINISHED":
                league.add_match(self,league_match(journey=d["matchday"]
                                        ,homeTeam=d["homeTeamName"]
                                        ,awayTeam=d["awayTeamName"]
                                        ,homeTeam_id=d["_links"]["homeTeam"]["href"][-3:]
                                        ,awayTeam_id=d["_links"]["awayTeam"]["href"][-3:]     
                                        ,date=d["date"][:10]
                                        ,goalsAwayTeam=d["result"]["goalsAwayTeam"]
                                        ,goalsHomeTeam=d["result"]["goalsHomeTeam"]
                                        ,league_id=self.league_id))

class amz_rds(object):
    
    
    def __init__(self,host,username,password,db,port):
        
        
        self.host=host
        self.username=username
        self.password=password
        self.db=db
        self.port=port
        self.cnx=None
        
    def open_cnx(self,state):
         
        if state==True:
            
            self.cnx=mysql.Connect(self.host,self.username,self.password, self.db,self.port)
        
        elif state==False:
            
            self.cnx=mysql.Connect(self.host,self.username,self.password, self.db,self.port).close()
            
    def add_match(self,cnx,journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_):
        
        cursor = cnx.cursor()
        
        add_match = ("INSERT INTO matchs_results "
                   "(journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_) "
                   "VALUES (%s, %s,%s, %s,%s, %s,%s)"
                    "ON DUPLICATE KEY UPDATE "
                    "journey=Values(journey),"
                    "homeTeam_id=VALUES(homeTeam_id),"
                    "awayTeam_id=VALUES(awayTeam_id),"
                    "goalsHomeTeam=VALUES(goalsHomeTeam),"
                    "goalsAwayTeam=VALUES(goalsAwayTeam),"
                    "date=VALUES(date),"
                    "index_=Values(index_);")   

        data_match = (journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_)

        cursor.execute(add_match, data_match)

        emp_no = cursor.lastrowid

        cnx.commit()

        cursor.close()
        
    def add_all_match(self,league):
        
        for m in league.matchs:
            
            self.add_match(self.cnx,m.journey,m.homeTeam_id,m.awayTeam_id,m.goalsHomeTeam,m.goalsAwayTeam,m.date,m.index)

            
            

In [69]:

ligue1=league(396)



rds=amz_rds(host= "chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com",
                  username="chimpdata",
                  password="godofdata",
                       port=8888,
                  db="chimpdata")

rds.open_cnx(True)

rds.add_all_match(ligue1)

rds.open_cnx(False)

In [48]:
print(rds.cnx)

In [241]:

db = mysql.connect(host= "chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com",
                  user="chimpdata",
                  passwd="godofdata",
                       port=8888,
                  db="chimpdata")
cursor = db.cursor()
cursor.execute("delete from ligue1_main")
cursor.close()
db.close()
print("done")
 

InternalError: (1205, 'Lock wait timeout exceeded; try restarting transaction')

In [224]:
cnx= {'host':'chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com',
  'username': 'chimpdata',
  'password': 'godofdata',
  'db': 'chimpdata'}


TABLES = {}
TABLES['matchs_results'] = (
    "CREATE TABLE matchs_results ("
    "journey varchar(2) NOT NULL,"
    "  homeTeam_id varchar(4) NOT NULL,"
    "  awayTeam_id varchar(4) NOT NULL,"
    "  goalsHomeTeam int NOT NULL,"
    "  goalsAwayTeam int NOT NULL,"
    "  date varchar(10) NOT NULL,"
    "  index_ varchar(15) NOT NULL,"
    "  PRIMARY KEY (index_))")
   

cnx=mysql.Connect(cnx['host'],cnx['username'],cnx['password'], cnx['db'],port=8888)
cursor = cnx.cursor()


cursor.execute(TABLES['matchs_results'])
cursor.close()
cnx.close()

In [70]:
cnx= {'host':'chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com',
  'username': 'chimpdata',
  'password': 'godofdata',
  'db': 'chimpdata'}


instruction  = "select * from matchs_results"
   

cnx=mysql.Connect(cnx['host'],cnx['username'],cnx['password'], cnx['db'],port=8888)
cursor = cnx.cursor()


cursor.execute(instruction)
cursor.close()
cnx.close()

In [74]:
cnx= {'host':'chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com',
  'username': 'chimpdata',
  'password': 'godofdata',
  'db': 'chimpdata'}

def add_match(journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_):
    
    cnx= {'host':'chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com',
  'username': 'chimpdata',
  'password': 'godofdata',
  'db': 'chimpdata'}
    
    cnx=mysql.Connect(cnx['host'],cnx['username'],cnx['password'], cnx['db'],port=8888)
    cursor = cnx.cursor()
    add_match = ("INSERT INTO matchs_results "
               "(journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_) "
               "VALUES (%s, %s,%s, %s,%s, %s,%s)"
                "ON DUPLICATE KEY UPDATE "
                "journey=Values(journey),"
                "homeTeam_id=VALUES(homeTeam_id),"
                "awayTeam_id=VALUES(awayTeam_id),"
                "goalsHomeTeam=VALUES(goalsHomeTeam),"
                "goalsAwayTeam=VALUES(goalsAwayTeam),"
                "date=VALUES(date),"
                "index_=Values(index_);")   

    data_match = (journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_)

    cursor.execute(add_match, data_match)
    emp_no = cursor.lastrowid

    cnx.commit()
  
    cursor.close()
    cnx.close()
    
def select_all(table,order):
    
    cnx= {'host':'chimpdata.cfoh6zvdxnyc.eu-west-1.rds.amazonaws.com',
  'username': 'chimpdata',
  'password': 'godofdata',
  'db': 'chimpdata'}
    
    cnx=mysql.Connect(cnx['host'],cnx['username'],cnx['password'], cnx['db'],port=8888)

    cursor = cnx.cursor()
    
    query = ("SELECT * FROM "+table+" order by "+order)

    cursor.execute(query)

    for a in cursor:
        print(a)

    cursor.close()
    cnx.close()

In [226]:
ligue1=league(396)

In [231]:
obj=ligue1.matchs[5]

add_match(obj.journey,obj.homeTeam_id,obj.awayTeam_id,obj.goalsHomeTeam,obj.goalsAwayTeam,obj.date,obj.index)


In [84]:
data=select_all('matchs_results','journey*1')

for yo in data:
    
    print(yo)

('1', '511', '527', 2, 1, '2015-08-09', '1511527396')
('1', '516', '514', 0, 1, '2015-08-08', '1516514396')
('1', '518', '532', 0, 2, '2015-08-08', '1518532396')
('1', '521', '524', 0, 1, '2015-08-07', '1521524396')
('1', '522', '548', 1, 2, '2015-08-08', '1522548396')
('1', '523', '525', 0, 0, '2015-08-09', '1523525396')
('1', '526', '547', 1, 2, '2015-08-09', '1526547396')
('1', '531', '555', 0, 0, '2015-08-08', '1531555396')
('1', '536', '529', 2, 1, '2015-08-08', '1536529396')
('1', '543', '538', 1, 0, '2015-08-08', '1543538396')
('2', '514', '511', 1, 0, '2015-08-15', '2514511396')
('2', '524', '555', 2, 0, '2015-08-16', '2524555396')
('2', '525', '536', 1, 1, '2015-08-16', '2525536396')
('2', '527', '526', 1, 1, '2015-08-15', '2527526396')
('2', '529', '518', 1, 0, '2015-08-15', '2529518396')
('2', '531', '522', 3, 3, '2015-08-15', '2531522396')
('2', '532', '543', 0, 0, '2015-08-15', '2532543396')
('2', '538', '523', 0, 1, '2015-08-15', '2538523396')
('2', '547', '516', 1, 0, '2

TypeError: 'NoneType' object is not iterable

In [217]:
ligue1.clubs

{'AS Monaco FC': '548',
 'AS Saint-Étienne': '527',
 'Angers SCO': '532',
 'EA Guingamp': '538',
 'ES Troyes AC': '531',
 'FC Girondins de Bordeaux': '526',
 'FC Lorient': '525',
 'FC Nantes': '543',
 'Gazélec Ajaccio': '555',
 'Montpellier Hérault SC': '518',
 'OGC Nice': '522',
 'OSC Lille': '521',
 'Olympique Lyonnais': '523',
 'Olympique de Marseille': '516',
 'Paris Saint-Germain': '524',
 'SC Bastia': '536',
 'SM Caen': '514',
 'Stade Rennais FC': '529',
 'Stade de Reims': '547',
 'Toulouse FC': '511'}

In [218]:
time.strftime("%Y-%M-%d")

'2016-54-26'

In [159]:
data=r.get(url,headers=headers)
data=data.json()
data

{'_links': {'self': {'href': 'http://api.football-data.org/v1/soccerseasons/396/fixtures'},
  'soccerseason': {'href': 'http://api.football-data.org/v1/soccerseasons/396'}},
 'count': 20,
 'teams': [{'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/521/fixtures'},
    'players': {'href': 'http://api.football-data.org/v1/teams/521/players'},
    'self': {'href': 'http://api.football-data.org/v1/teams/521'}},
   'code': 'OSC',
   'crestUrl': 'http://upload.wikimedia.org/wikipedia/de/0/08/LOSC_Lille_Crest_2012.png',
   'name': 'OSC Lille',
   'shortName': 'Lille',
   'squadMarketValue': '67,700,000 €'},
  {'_links': {'fixtures': {'href': 'http://api.football-data.org/v1/teams/524/fixtures'},
    'players': {'href': 'http://api.football-data.org/v1/teams/524/players'},
    'self': {'href': 'http://api.football-data.org/v1/teams/524'}},
   'code': 'PSG',
   'crestUrl': 'http://upload.wikimedia.org/wikipedia/de/1/1e/Paris_Saint-Germain_(seit_2013).gif',
   'name': 'Pari

In [160]:
for d in data:
    print(d["caption"])
    print(d["id"])

TypeError: string indices must be integers

In [111]:
self.journey=d["matchday"]
        self.homeTeam=d["homeTeam"]
        self.awayTeam=d["awayTeam"]
        self.date=d["date"]
        self.goalsAwayTeam=d["result"]["goalsAwayTeam"]
        self.goalsHomeTeam=d["result"]["goalsHomeTeam"]
(self,journey=d["matchday"],homeTeam=homeTeam=d["homeTeam"],awayTeam=["awayTeam"],date=d["date"],goalsAwayTeam=d["result"]["goalsAwayTeam"],goalsHomeTeam=d["result"]["goalsHomeTeam"])

IndentationError: unexpected indent (<ipython-input-111-78369d941d39>, line 2)

In [121]:
key="9a0d6c52a1fc405f8d02e5c08d09a2df"
url='http://api.football-data.org/v1/soccerseasons/396/fixtures'
headers = {"X-Auth-Token": key}
d1_id=396
data=r.get(url,headers=headers)
data=data.json()
for d in data['fixtures']:
    print(d)


{'date': '2015-08-07T18:30:00Z', 'homeTeamName': 'OSC Lille', 'status': 'FINISHED', 'result': {'goalsAwayTeam': 1, 'goalsHomeTeam': 0}, 'awayTeamName': 'Paris Saint-Germain', '_links': {'self': {'href': 'http://api.football-data.org/v1/fixtures/146351'}, 'soccerseason': {'href': 'http://api.football-data.org/v1/soccerseasons/396'}, 'homeTeam': {'href': 'http://api.football-data.org/v1/teams/521'}, 'awayTeam': {'href': 'http://api.football-data.org/v1/teams/524'}}, 'matchday': 1}
{'date': '2015-08-08T19:00:00Z', 'homeTeamName': 'SC Bastia', 'status': 'FINISHED', 'result': {'goalsAwayTeam': 1, 'goalsHomeTeam': 2}, 'awayTeamName': 'Stade Rennais FC', '_links': {'self': {'href': 'http://api.football-data.org/v1/fixtures/146348'}, 'soccerseason': {'href': 'http://api.football-data.org/v1/soccerseasons/396'}, 'homeTeam': {'href': 'http://api.football-data.org/v1/teams/536'}, 'awayTeam': {'href': 'http://api.football-data.org/v1/teams/529'}}, 'matchday': 1}
{'date': '2015-08-08T19:00:00Z', 'h